In [4]:
import torch
import os
import numpy as np
import random 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 31
seed_everything(SEED)

import flair
flair.device = torch.device('cuda:2') 

In [5]:
from oracle import HybridOracle
from active_learner import ActiveLearner

In [6]:
EXPERIMENT = 'absolute_sampling'

In [7]:
oracle = HybridOracle(
    experiment_name=EXPERIMENT,
    all_data_file='../twitter_dataset/train_10.csv',
    valid_file='../twitter_dataset/valid.csv',
    test_file='../twitter_dataset/test.csv'
)

../twitter_dataset/train_10.csv
Starting with labelled data


In [8]:
sentences = oracle.get_all_sentences()

In [9]:
learner = ActiveLearner(sentences,
                        experiment_name=EXPERIMENT,
                        oracle=oracle, 
                        embeddings_storage_mode='gpu')

In [10]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.data import Dictionary, Sentence

In [11]:
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                    FlairEmbeddings('news-forward'),
                    FlairEmbeddings('news-backward'),
                   ]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

label_dict = Dictionary()
label_dict.idx2item = [b'0', b'1']
label_dict.item2idx = {b'0': 0, b'1': 1}
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [12]:
classifier = classifier.load('../twitter_dataset/0_1/best-model.pt')

2019-10-19 13:35:30,051 loading file ../twitter_dataset/0_1/best-model.pt


In [13]:
classifier, opt_state = learner.step(classifier, step_num=0, max_sample_size=100000, labelling_step_size=128, step_lr=1e-5, sampling_method='absolute')

STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12/128 LABELS
DONE 13/128 LABELS
DONE 14/128 LABELS
DONE 15/128 LABELS
DONE 16/128 LABELS
DONE 17/128 LABELS
DONE 18/128 LABELS
DONE 19/128 LABELS
DONE 20/128 LABELS
DONE 21/128 LABELS
DONE 22/128 LABELS
DONE 23/128 LABELS
DONE 24/128 LABELS
DONE 25/128 LABELS
DONE 26/128 LABELS
DONE 27/128 LABELS
DONE 28/128 LABELS
DONE 29/128 LABELS
DONE 30/128 LABELS
DONE 31/128 LABELS
DONE 32/128 LABELS
DONE 33/128 LABELS
DONE 34/128 LABELS
DONE 35/128 LABELS
DONE 36/128 LABELS
DONE 37/128 LABELS
DONE 38/128 LABELS
DONE 39/128 LABELS
DONE 40/128 LABELS
DONE 41/128 LABELS
DONE 42/128 LABELS
DONE 43/128 LABELS
DONE 44/128 LABELS
DONE 45/128 LABELS
DONE 46/128 LABELS
DONE 47/128 LABELS
DONE 48/128 LABELS
DONE 49/128 LABELS
DONE 50/128 LABELS
DONE 51/128 LABELS
DONE 52/128 LABELS
DO

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:37:59,948 ----------------------------------------------------------------------------------------------------
2019-10-19 13:37:59,949 Testing using best model ...
2019-10-19 13:37:59,951 loading file absolute_sampling/0/best-model.pt
2019-10-19 13:39:18,442 0.7014	0.7015	0.7014
2019-10-19 13:39:18,444 
MICRO_AVG: acc 0.5402 - f1-score 0.7014
MACRO_AVG: acc 0.5399 - f1-score 0.7011499999999999
0          tp: 3674 - fp: 1669 - fn: 1316 - tn: 3342 - precision: 0.6876 - recall: 0.7363 - accuracy: 0.5517 - f1-score: 0.7111
1          tp: 3341 - fp: 1317 - fn: 1669 - tn: 3674 - precision: 0.7173 - recall: 0.6669 - accuracy: 0.5281 - f1-score: 0.6912
2019-10-19 13:39:18,444 ----------------------------------------------------------------------------------------------------


In [ ]:
for step_num in range(1, 100):
    classifier, opt_state = learner.step(classifier,
                                         step_num=step_num,
                                         optimizer_state=opt_state,
                                         max_sample_size=100000, 
                                         labelling_step_size=128,
                                         step_lr=1e-5, 
                                         sampling_method='absolute')

STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12/128 LABELS
DONE 13/128 LABELS
DONE 14/128 LABELS
DONE 15/128 LABELS
DONE 16/128 LABELS
DONE 17/128 LABELS
DONE 18/128 LABELS
DONE 19/128 LABELS
DONE 20/128 LABELS
DONE 21/128 LABELS
DONE 22/128 LABELS
DONE 23/128 LABELS
DONE 24/128 LABELS
DONE 25/128 LABELS
DONE 26/128 LABELS
DONE 27/128 LABELS
DONE 28/128 LABELS
DONE 29/128 LABELS
DONE 30/128 LABELS
DONE 31/128 LABELS
DONE 32/128 LABELS
DONE 33/128 LABELS
DONE 34/128 LABELS
DONE 35/128 LABELS
DONE 36/128 LABELS
DONE 37/128 LABELS
DONE 38/128 LABELS
DONE 39/128 LABELS
DONE 40/128 LABELS
DONE 41/128 LABELS
DONE 42/128 LABELS
DONE 43/128 LABELS
DONE 44/128 LABELS
DONE 45/128 LABELS
DONE 46/128 LABELS
DONE 47/128 LABELS
DONE 48/128 LABELS
DONE 49/128 LABELS
DONE 50/128 LABELS
DONE 51/128 LABELS
DONE 52/128 LABELS
DO

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:41:51,373 ----------------------------------------------------------------------------------------------------
2019-10-19 13:41:51,376 Testing using best model ...
2019-10-19 13:41:51,379 loading file absolute_sampling/1/best-model.pt
2019-10-19 13:43:02,461 0.6915	0.6916	0.6915
2019-10-19 13:43:02,462 
MICRO_AVG: acc 0.5285 - f1-score 0.6915
MACRO_AVG: acc 0.5266 - f1-score 0.68935
0          tp: 3881 - fp: 1975 - fn: 1109 - tn: 3036 - precision: 0.6627 - recall: 0.7778 - accuracy: 0.5572 - f1-score: 0.7157
1          tp: 3035 - fp: 1110 - fn: 1975 - tn: 3881 - precision: 0.7322 - recall: 0.6058 - accuracy: 0.4959 - f1-score: 0.6630
2019-10-19 13:43:02,463 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS


/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:45:34,543 ----------------------------------------------------------------------------------------------------
2019-10-19 13:45:34,544 Testing using best model ...
2019-10-19 13:45:34,545 loading file absolute_sampling/2/best-model.pt
2019-10-19 13:46:55,146 0.6808	0.6809	0.6808
2019-10-19 13:46:55,148 
MICRO_AVG: acc 0.5161 - f1-score 0.6808
MACRO_AVG: acc 0.5109 - f1-score 0.67475
0          tp: 4092 - fp: 2293 - fn: 898 - tn: 2718 - precision: 0.6409 - recall: 0.8200 - accuracy: 0.5619 - f1-score: 0.7195
1          tp: 2717 - fp: 899 - fn: 2293 - tn: 4092 - precision: 0.7514 - recall: 0.5423 - accuracy: 0.4598 - f1-score: 0.6300
2019-10-19 13:46:55,149 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DO

...

2019-10-19 14:39:40,649 ----------------------------------------------------------------------------------------------------
2019-10-19 14:39:40,650 Testing using best model ...
2019-10-19 14:39:40,652 loading file absolute_sampling/15/best-model.pt
2019-10-19 14:40:57,195 0.5223	0.5224	0.5223
2019-10-19 14:40:57,196 
MICRO_AVG: acc 0.3535 - f1-score 0.5223
MACRO_AVG: acc 0.2808 - f1-score 0.387
0          tp: 4962 - fp: 4748 - fn: 28 - tn: 263 - precision: 0.5110 - recall: 0.9944 - accuracy: 0.5096 - f1-score: 0.6751
1          tp: 262 - fp: 29 - fn: 4748 - tn: 4962 - precision: 0.9003 - recall: 0.0523 - accuracy: 0.0520 - f1-score: 0.0989
2019-10-19 14:40:57,197 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:43:22,065 ----------------------------------------------------------------------------------------------------
2019-10-19 14:43:22,066 Testing using best model ...
2019-10-19 14:43:22,067 loading file absolute_sampling/16/best-model.pt
2019-10-19 14:44:31,449 0.5173	0.5174	0.5173
2019-10-19 14:44:31,451 
MICRO_AVG: acc 0.349 - f1-score 0.5173
MACRO_AVG: acc 0.2742 - f1-score 0.376
0          tp: 4967 - fp: 4803 - fn: 23 - tn: 208 - precision: 0.5084 - recall: 0.9954 - accuracy: 0.5072 - f1-score: 0.6730
1          tp: 207 - fp: 24 - fn: 4803 - tn: 4967 - precision: 0.8961 - recall: 0.0413 - accuracy: 0.0411 - f1-score: 0.0790
2019-10-19 14:44:31,452 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12/

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:46:54,568 ----------------------------------------------------------------------------------------------------
2019-10-19 14:46:54,569 Testing using best model ...
2019-10-19 14:46:54,570 loading file absolute_sampling/17/best-model.pt
2019-10-19 14:48:10,992 0.5145	0.5146	0.5145
2019-10-19 14:48:10,994 
MICRO_AVG: acc 0.3464 - f1-score 0.5145
MACRO_AVG: acc 0.2704 - f1-score 0.36955000000000005
0          tp: 4971 - fp: 4835 - fn: 19 - tn: 176 - precision: 0.5069 - recall: 0.9962 - accuracy: 0.5060 - f1-score: 0.6719
1          tp: 175 - fp: 20 - fn: 4835 - tn: 4971 - precision: 0.8974 - recall: 0.0349 - accuracy: 0.0348 - f1-score: 0.0672
2019-10-19 14:48:10,995 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 